## Day 35 Lecture 2 Assignment

In this assignment, we will combine what we have learned so far about classification algorithms this week.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
admission = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Admission_Predict.csv')

In [3]:
admission.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In this assignment, we will predict the probability of a student being admitted to a PhD program given their stats. To make the predictions, find the median for the Chance of Admit column. Create an admit column where all probabilities above the median will receive a 1 for that column and all probabilities below the median will be a zero.

Below you will process and clean the data, try the SVM classifier, the gradient boosted decision tree classifier and XGBoost, and compare your results.

Have fun!

In [4]:
admission.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [5]:
admission.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,200.500000,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500,0.724350
std,115.614301,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362,0.142609
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000,0.340000
25%,100.750000,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000,0.640000
50%,200.500000,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000,0.730000
75%,300.250000,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000,0.830000
max,400.000000,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,0.970000


In [6]:
# Answer below:
admission2 = admission.copy()
admission2['Above_Median'] = np.where(admission2['Chance of Admit '] > admission.median(axis=0)[8], 1, 0)
admission2

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit,Above_Median
0,1,337,118,4,4.5,4.5,9.65,1,0.92,1
1,2,324,107,4,4.0,4.5,8.87,1,0.76,1
2,3,316,104,3,3.0,3.5,8.00,1,0.72,0
3,4,322,110,3,3.5,2.5,8.67,1,0.80,1
4,5,314,103,2,2.0,3.0,8.21,0,0.65,0
...,...,...,...,...,...,...,...,...,...,...
395,396,324,110,3,3.5,3.5,9.04,1,0.82,1
396,397,325,107,3,3.0,3.5,9.11,1,0.84,1
397,398,330,116,4,5.0,4.5,9.45,1,0.91,1
398,399,312,103,3,3.5,4.0,8.78,0,0.67,0


In [7]:
admission2.isnull().sum().sum()

0

In [8]:
admission2.Above_Median.value_counts()

0    209
1    191
Name: Above_Median, dtype: int64

In [29]:
from sklearn.metrics import classification_report
import time

def results(model):
    time_start = time.time()
    print('Train Accuracy:', model.score(X_train, y_train), 
          '| Test Accuracy:', model.score(X_test, y_test), 
          '| Execution time: {}'.format(time.time() - time_start))
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    res_train = y_train - y_train_pred
    res_test = y_test - y_test_pred
    print(classification_report(y_test, y_test_pred))
    display(pd.crosstab(y_test, y_test_pred))

In [30]:
#SVM Classifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

y = admission2.Above_Median
X = admission2.drop(columns=['Above_Median', 'Chance of Admit '])

X_train, X_test, y_train, y_test = train_test_split(X, y)

svc = SVC()
svc.fit(X_train, y_train)
results(svc)

Train Accuracy: 0.7666666666666667 | Test Accuracy: 0.8 | Execution time: 0.01719951629638672
              precision    recall  f1-score   support

           0       0.79      0.87      0.83        55
           1       0.82      0.71      0.76        45

    accuracy                           0.80       100
   macro avg       0.80      0.79      0.79       100
weighted avg       0.80      0.80      0.80       100



col_0,0,1
Above_Median,,
0,48,7
1,13,32


In [31]:
from sklearn.ensemble import GradientBoostingClassifier

y = admission2.Above_Median
X = admission2.drop(columns=['Above_Median', 'Chance of Admit '])

X_train, X_test, y_train, y_test = train_test_split(X, y)

gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)
results(gbc)

Train Accuracy: 0.9966666666666667 | Test Accuracy: 0.84 | Execution time: 0.006266117095947266
              precision    recall  f1-score   support

           0       0.78      0.89      0.83        45
           1       0.90      0.80      0.85        55

    accuracy                           0.84       100
   macro avg       0.84      0.84      0.84       100
weighted avg       0.85      0.84      0.84       100



col_0,0,1
Above_Median,,
0,40,5
1,11,44


In [16]:
!pip install xgboost

In [32]:
from xgboost import XGBClassifier

y = admission2.Above_Median
X = admission2.drop(columns=['Above_Median', 'Chance of Admit '])

X_train, X_test, y_train, y_test = train_test_split(X, y)

xgb = XGBClassifier() 
xgb.fit(X_train, y_train)
results(xgb)

Train Accuracy: 1.0 | Test Accuracy: 0.86 | Execution time: 0.02619194984436035
              precision    recall  f1-score   support

           0       0.82      0.92      0.87        49
           1       0.91      0.80      0.85        51

    accuracy                           0.86       100
   macro avg       0.86      0.86      0.86       100
weighted avg       0.87      0.86      0.86       100



col_0,0,1
Above_Median,,
0,45,4
1,10,41
